In [2]:
import contractions
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
import pandas as pd
import contractions
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence = contractions.fix(sentence)
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(lemma_words)
        


# Process for all messages
df_gender = pd.read_csv(r"big.csv",encoding='latin1')
preprocessed = []
for i in range(0,9499):
    message = df_gender['Content'].iloc[i]
    processed_Text = preprocess(message)
    df_gender.loc[i, 'Content'] = processed_Text
    preprocessed.append(str(processed_Text))

# Process for all messages
df_age = pd.read_csv(r"age.csv",encoding='latin1')
preprocessed = []
for i in range(0,9499):
    message = df_age['Content'].iloc[i]
    processed_Text = preprocess(message)
    df_age.loc[i, 'Content'] = processed_Text
    preprocessed.append(str(processed_Text))

In [ ]:
from sklearn.naive_bayes import MultinomialNB 
from sklearn.model_selection import GridSearchCV 
from sklearn.feature_extraction.text import TfidfVectorizer 


NB = MultinomialNB()
parameters = {'alpha':  [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 2500, 5000, 10000]}
classifier = GridSearchCV(NB, parameters, cv=3, scoring='accuracy',return_train_score=True)
vectz = TfidfVectorizer(norm = False, smooth_idf = False)
X_tr_bow = vectz.fit_transform(X_train)
classifier.fit(X_tr_bow, y_train)


In [3]:
df_age.head(5)

,age,FileName,Content,Unnamed: 3
0,1,0000e06e07496624211632e8e264126c.txt,paranoia settin sumbuddi go get hurt real bad ...,NaN
1,1,000235a2ba2f48231b7d24e1f08d7878.txt,damn game kick as today huge water fight phone...,NaN
2,1,000c4b6e2468f7d528876fd1a6dffd4c.txt,better conquer win thousand battl victori cann...,NaN
3,1,001187432d2a247562082cd0000dec40.txt,hot lol aloha everyon head soon see everyon ge...,NaN
4,1,001494c3b74f124a2e3435fff17f376b.txt,lord earth maker thing alon one true god make ...,NaN


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_gender, y_test_gender = train_test_split(df_age['Content'],df_gender['gender'], random_state=42, test_size=0.1)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_age, y_test_age = train_test_split(df_age['Content'],df_age['age'], random_state=42, test_size=0.1)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


logreg_age = Pipeline([('vect', TfidfVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=2, C=1e5)),
               ])
logreg_gender = Pipeline([('vect', TfidfVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=2, C=1e5)),
               ])

logreg_age.fit(X_train, y_train_age)
logreg_gender.fit(X_train, y_train_gender)
y_pred_age = logreg_age.predict(X_test)
y_pred_gender = logreg_gender.predict(X_test)

print('accuracy age %s' % accuracy_score(y_pred_age, y_test_age))
print('accuracy gender %s' % accuracy_score(y_pred_gender, y_test_gender))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

accuracy age 0.6221052631578947
accuracy gender 0.7368421052631579


In [10]:
import pickle
pickle.dump(logreg_age, open('model_age_2.pkl', 'wb'))

In [11]:
import pickle
pickle.dump(logreg_gender, open('model_gender_3.pkl', 'wb'))